# Visualization of Example 1 & 2 from the sola paper 

This is a python notebook for visualizing the connection between lie group mapping, using a slider to change the value of the parameters.
For that we will use some python libraries like `numpy`, `matplotlib` and `ipywidgets` and `manifpy`.

In [ ]:
from manifpy import *
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

## 2d plot of the example 1 from the paper

The following shows the $S^1$ (the unit complex numbers).
In our code we define the SO2Tangent and use exp and log to map between the lie group and the lie algebra. The exp function wraps the tangent vector over the surface without changing its length. Similarly the log function unwraps the tangent vector from the surface to the tangent space.

In [ ]:
def update(val):
    dstate = SO2Tangent(val) 
    
    state = dstate.exp()
    
    dot.set_data([state.real()], [state.imag()])
    line.set_data([0, state.real()], [0, state.imag()])
    red_line.set_data([1, 1], [0, val])
    text.set_text(f'z = [{state.coeffs()[0]:.2f}, {state.coeffs()[1]:.2f}]')
    
    theta_value = val
    theta_text.set_text(f'\u03B8 = {theta_value:.2f}')
    
    plt.draw()

fig, ax = plt.subplots(figsize=(8, 9))
plt.subplots_adjust(bottom=0.25)
ax.set_title("Example 1 Unit complex numbers group")
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), color='grey')

dstate = SO2Tangent(1)
state = dstate.exp()

line, = ax.plot([0, state.real()], [0, state.imag()], 'k-')  
ax.plot([0, 1], [0, 0], 'k-')  
red_line, = ax.plot([1, 1], [0, dstate.coeffs()[0]], 'r-', label=r'$\theta$') 

dot, = ax.plot(state.real(), state.imag(), 'ro', label='z')
text = ax.text(0.5, 1.2, f'z = [{state.coeffs()[0]:.2f}, {state.coeffs()[1]:.2f}]', verticalalignment='bottom', horizontalalignment='right')

theta_value = dstate.coeffs()[0]
theta_text = ax.text(1.2, 1, f'\u03B8 = {theta_value}', verticalalignment='bottom')

slider_w = 0.65
slider_h = 0.03
slider_l = (1 - slider_w) / 2
slider_b = 0.1
ax_th = plt.axes([slider_l, slider_b, slider_w, slider_h])
slider_re = Slider(ax_th, 'Theta', -np.pi, np.pi, valinit=dstate.coeffs()[0])

slider_re.on_changed(update)
slider_re.valmin =-np.pi
slider_re.valmax = np.pi

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_aspect('equal')
ax.legend(loc='lower left')
ax.set_xlim(-2.3, 2.3)
ax.set_ylim(-2.3, 2.3)
ax.grid(True)
plt.show()

We have the constraint that

$$
\begin{align*}
z^{\top} z &= 1 \\
\dot{z}^{\top}z + z^{\top}\dot{z} &= 0 \\
z^{\top}\dot{z} &= - \dot{z}^{\top}z = - (z^{\top}\dot{z})^{\top}
\end{align*}
$$
Which is only true if 
$$
\begin{align*}
\dot{z} = z \cdot i \omega
\end{align*}
$$
And in our case
$$
z^{\top} = 1
$$
So the derivative is purely imaginary. Meaning that it is on the red line

## 3d plot of example 2 from the paper

The following shows the $S^3$ (the unit quaternions group).
In our code we define the SO3Tangent and use exp and log to map between the lie group and the lie algebra. The exp function wraps the tangent vector over the surface without changing its length. Similarly the log function unwraps the tangent vector from the surface to the tangent space. We needed to make changes since the library defines things in the NED frame while our visualization is in the ENU frame. 

In [ ]:
def update(val):
    theta1 = slider_theta1.val
    theta2 = slider_theta2.val
    dstate = SO3Tangent(np.array([0, -theta2, theta1]))
    state = dstate.exp()
    R = state.rotation()
    x, y, z = R[:, 0]
    
    line.set_data([0, x], [0, y])
    line.set_3d_properties([0, z])
    
    red_line.set_data([1, 1], [0, theta1])
    red_line.set_3d_properties([0, theta2])
    
    dot.set_data([x], [y])
    dot.set_3d_properties([z])
    
    text.set_text(f'z = [{x:.2f}, {y:.2f}, {z:.2f}]')
    
    theta_text.set_text(f'\u03B8 = [0, {theta1:.2f}, {theta2:.2f}]')
    
    plt.draw()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_title("3D Example with the unit quaternion group")

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z, color='b', alpha=0.2)

x_plane = np.ones(100)
y_plane = np.linspace(-1, 1, 100)
z_plane = np.linspace(-1, 1, 100)
Y_plane, Z_plane = np.meshgrid(y_plane, z_plane)
X_plane = np.ones_like(x_plane)
ax.plot_surface(X_plane, Y_plane, Z_plane, color='r', alpha=0.2)

theta_initial_val = 1
red_line, = ax.plot([1, 1], [0, theta_initial_val], [0, theta_initial_val], 'r-', label=r'$\theta$')
ax.plot([0, 1], [0, 0], [0, 0], 'k-')  

dstate = SO3Tangent(np.array([0, -1, 1]))
state = dstate.exp()
R = state.rotation()
x, y, z = R[:, 0]
dot, = ax.plot([x], [y], [z], 'ro', label='z')
line, = ax.plot([0, x], [0, y], [0, z], 'k-')

text = ax.text(0.5, 1.1, 1.5, '', verticalalignment='bottom', horizontalalignment='right')
theta_text = ax.text(0.5, 1.3, 1.1, '', verticalalignment='bottom', horizontalalignment='right')

slider_w = 0.65
slider_h = 0.03
slider_l = (1 - slider_w) / 2
slider_b = 0.10

ax_theta1 = plt.axes([slider_l, slider_b, slider_w, slider_h])
ax_theta2 = plt.axes([slider_l, slider_b - 0.05, slider_w, slider_h])
slider_theta1 = Slider(ax_theta1, 'Theta1', -np.pi, np.pi, valinit=theta_initial_val)
slider_theta2 = Slider(ax_theta2, 'Theta2', -np.pi, np.pi, valinit=theta_initial_val)

slider_theta1.on_changed(update)
slider_theta2.on_changed(update)

ax.set_box_aspect([1,1,1])
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.legend(loc='lower left')
ax.set_xlim(-2.3, 2.3)
ax.set_ylim(-2.3, 2.3)
ax.set_zlim(-2.3, 2.3)
plt.show()